# Interactive Plotting with Jupyter

There are several ways to interactively plot. In this tutorial I will show how to interact with imaging and spectral data.  There are other ways to interact with large tables of data using either [Bokeh](https://docs.bokeh.org/en/latest/index.html) (shown the Skyfit notebook) or [Glue](http://docs.glueviz.org/en/stable). A non-python based solution that also works with large tables of data is Topcat. 

Most of the methods here will work on the command line.  In order to make this work within Jupyter you will need the following modules.

conda install -c conda-forge ipympl

conda install -c conda-forge ipywidgets

https://ipywidgets.readthedocs.io/


In [1]:
import glob,os,sys

import numpy as np
import matplotlib.pyplot as plt 

import astropy.io.fits as pyfits
import astropy.units as u
from astroquery.skyview import SkyView

import ipywidgets as widgets

# https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html

Here we need an image to play with

In [2]:
ext = 0

# download an image
pflist = SkyView.get_images(position='M82', survey=['SDSSr'], radius=10 * u.arcmin)
pf = pflist[0] # first element of the list, might need a loop if multiple images

# or load an image
#pf = pyfits.open('m82.fits')

image = pf[ext].data

Next we need to turn on the interactive plotting.

In [3]:
# turn-on interactive plots
%matplotlib widget 

# Display an image

In [4]:
#plt.ioff()
fig = plt.figure(figsize=[6,6])
plt.ion()

p = fig.add_subplot(111)
p.imshow(image, interpolation='Nearest', origin='lower', vmin=-10, vmax=20, cmap='viridis')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Add an event to the display

There are several types of matplotlib events that you can use to interact with a figure. 

A few useful events are the following:

'button_press_event' 	
'button_release_event' 	
'key_press_event'  
'key_release_event'  

For more information on event handling and examples check out the following website: 
https://matplotlib.org/stable/users/event_handling.html

In [8]:
#plt.ioff()
fig = plt.figure(figsize=[6,6])
plt.ion()

p = fig.add_subplot(111)
p.imshow(image, interpolation='Nearest', origin='lower', vmin=-10, vmax=20, cmap='viridis')

def on_key_press(event):
    xc, yc = event.xdata, event.ydata

    if event.key == 'm':
        p.plot(xc,yc,'ro', markersize=5)
        fig.canvas.draw_idle

fig.canvas.mpl_connect('key_press_event', on_key_press)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Add output to the display with the event

In [9]:
# https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html

#plt.ioff()
fig = plt.figure(figsize=[6,6])
plt.ion()

p = fig.add_subplot(111)
p.imshow(image, interpolation='Nearest', origin='lower', vmin=-10, vmax=20, cmap='viridis')

out = widgets.Output()
@out.capture()
def on_key_press(event):
    xc, yc = event.xdata, event.ydata

    if event.key == 'm':
        p.plot(xc,yc,'ro', markersize=5)
        fig.canvas.draw_idle
        
        print("[%.2f, %.2f]" % (xc,yc))

fig.canvas.mpl_connect('key_press_event', on_key_press)

display(out)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

In [7]:
# ensure we are interactive mode 
# this is default but if this notebook is executed out of order it may have been turned off

out = widgets.Output()
with out:
    fig = plt.figure()

ax = fig.gca()
ax.imshow(image)

widgets.AppLayout(
    center=out,
    footer=widgets.Button(icon='check'),
    pane_heights=[0, 6, 1]
)

AppLayout(children=(Button(icon='check', layout=Layout(grid_area='footer'), style=ButtonStyle()), Output(layou…

We can also write a Python class, this makes it more convient for dealing with multiple interactive events (i.e. keypress, mouse clicking, dragging, etc).

In [ ]:
class GUI_inter:

    def __init__(self,fig,img):
        self.fig = fig
        self.p = self.fig.gca()

        self.img = img

        self.display()

    def display(self,sigma=10.0):
        plt.clf()

        self.v0 = np.mean(self.img) - sigma * np.std(self.img)
        self.v1 = np.mean(self.img) + sigma * np.std(self.img)

        self.p = self.fig.add_subplot(111)
        self.p.imshow(self.img, interpolation='Nearest', origin='lower', 
                      vmin=self.v0, vmax=self.v1, cmap='viridis')
        plt.draw()

    def on_key_press(self, event):
        xc, yc = event.xdata, event.ydata

        if event.key == 'm':
            self.p.plot(xc,yc,'bo', markersize=5)
            fig.canvas.draw_idle

            print("[%.2f, %.2f]" % (xc,yc))

In [ ]:
fig = plt.figure(figsize=[6,6])
G = GUI_inter(fig, image)
fig.canvas.mpl_connect('key_press_event', G.on_key_press)

In [ ]:
# turn-off interactive plots
%matplotlib inline 